In [1]:
import numpy as np

from db_comm import PortfolioDBManager, DATABASE_NAME
from datetime import datetime

In [2]:
pm = PortfolioDBManager(DATABASE_NAME)

Deposit cash

In [26]:
pm.deposit_cash(200)

✅ Recorded DEPOSIT: 200 CASH @ 1.0. Snapshot updated.


View portfolio

In [4]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,CASH,99.3112,1.000,99.311200
1,AAPL,0.0010,688.758,0.688758


Insufuffcient cash for transaction

In [5]:
pm.record_transaction(
    tx_type="BUY",
    ticker="AAPL",
    shares=1,
    actual_price=150,
)

❌ Insufficient cash balance to BUY 150 of AAPL. Current CASH: 99.3112


'Transaction Denied: Insufficient Cash'

Calculation Motor

In [6]:
from portfolio_test.motor import CalculationMotor

In [7]:
cm = CalculationMotor("AAPL")

In [8]:
cm.df

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Price_SEK
Date,,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.468262,135480400,0.0,0.0,688.757977
2020-01-03,74.287498,75.144997,74.125000,74.357498,71.763718,146322800,0.0,0.0,690.873736
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.335564,118387200,0.0,0.0,683.061770
2020-01-07,74.959999,75.224998,74.370003,74.597504,71.995369,108872000,0.0,0.0,697.127991
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.153488,132079200,0.0,0.0,690.897009
...,...,...,...,...,...,...,...,...,...
2025-12-09,278.160004,280.029999,276.920013,277.179993,277.179993,32193300,0.0,0.0,2586.888034
2025-12-10,277.750000,279.750000,276.440002,278.779999,278.779999,33038300,0.0,0.0,2583.075000
2025-12-11,279.100006,279.589996,273.809998,278.029999,278.029999,33248000,0.0,0.0,2595.630057


Test buying aapl stock

In [9]:
oldest_date = cm.df.index.min()
cm.df.loc[oldest_date, "Price_SEK"]

np.float64(688.757977294922)

In [10]:
cm.df.Price_SEK.loc[oldest_date]

np.float64(688.757977294922)

In [16]:
pm.record_transaction(
    tx_type="BUY",
    ticker="AAPL",
    shares=.001,
    actual_price=cm.df.Price_SEK.loc[oldest_date],
    tx_datetime=oldest_date,
    currency="USD",
)

   -> Cash balance adjusted by -0.6888
✅ Recorded BUY: 0.001 AAPL @ 688.758. Snapshot updated.


In [25]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,AAPL,0.146391,683.06177,99.994262


In [12]:
pm.get_cash_balance()

99.3112

Build calculation motor for AAPL

In [49]:
import numpy as np

for date in cm.df.index:
    # if price today is lower than yesterday, buy as much as possible
    cm.df['Price_SEK_ydy'] = cm.df['Price_SEK'].shift(1)
    if date != cm.df.index.min():
        #print(f"Processing date: {date.date()}")
        price = cm.df.Price_SEK.loc[date] # price in SEK
        price_yesterday = cm.df.Price_SEK_ydy.loc[date]
        if price < price_yesterday:
            #print("Price dropped, buying AAPL")
            cash_balance = pm.get_cash_balance()
            shares_to_buy = np.divide(cash_balance, price)
            if shares_to_buy > 0:
                print("Buying AAPL")
                pm.record_transaction(
                    tx_type="BUY",
                    ticker="AAPL",
                    shares=shares_to_buy,
                    actual_price=price,
                    tx_datetime=date,
                    currency="SEK",
                )
        # if price today is higher than yesterday, sell all shares
        elif price > price_yesterday:
            portfolio = pm.get_portfolio_snapshot()
            if "AAPL" in list(portfolio["ticker"]):
                shares_to_sell = portfolio[portfolio["ticker"]=="AAPL"]["net_shares"].iloc[0]
                if shares_to_sell > 0:
                    pm.record_transaction(
                        tx_type="SELL",
                        ticker="AAPL",
                        shares=shares_to_sell,
                        actual_price=price,
                        tx_datetime=date,
                        currency="SEK",
                    )


   -> Cash balance adjusted by 101.1379
✅ Recorded SELL: 0.14639124334149933 AAPL @ 690.8737358093263. Snapshot updated.
Buying AAPL
   -> Cash balance adjusted by -301.1379
✅ Recorded BUY: 0.44086481382007364 AAPL @ 683.0617698669435. Snapshot updated.
   -> Cash balance adjusted by 307.3392
✅ Recorded SELL: 0.44086481382007364 AAPL @ 697.1279914855958. Snapshot updated.
Buying AAPL
   -> Cash balance adjusted by -307.3392
✅ Recorded BUY: 0.4448408318641495 AAPL @ 690.8970085144043. Snapshot updated.
   -> Cash balance adjusted by 317.7645
✅ Recorded SELL: 0.4448408318641495 AAPL @ 714.3329772949219. Snapshot updated.
Buying AAPL
   -> Cash balance adjusted by -317.7645
✅ Recorded BUY: 0.4382648726979315 AAPL @ 725.0512641906739. Snapshot updated.
   -> Cash balance adjusted by 319.5375
✅ Recorded SELL: 0.4382648726979315 AAPL @ 729.0967414855958. Snapshot updated.
Buying AAPL
   -> Cash balance adjusted by -319.5375
✅ Recorded BUY: 0.43229579160233605 AAPL @ 739.1640312194825. Snapsh

In [50]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,CASH,576.7228,1.0,576.7228


In [64]:
# find Price_SEK at max date
last_price = cm.df.loc[cm.df.index.max(), "Price_SEK"]
earnings = pm.get_portfolio_snapshot()["total_position_value"].sum()

if earnings < last_price:
    print("Overall loss")
else:
    print("Overall gain")

Overall loss


In [52]:
cm.df[["Price_SEK", "Price_SEK_ydy"]]

,Price_SEK,Price_SEK_ydy
Date,,
2020-01-02,688.757977,NaN
2020-01-03,690.873736,688.757977
2020-01-06,683.061770,690.873736
2020-01-07,697.127991,683.061770
2020-01-08,690.897009,697.127991
...,...,...
2025-12-09,2586.888034,2586.609045
2025-12-10,2583.075000,2586.888034
2025-12-11,2595.630057,2583.075000


In [55]:
df = cm.df

In [57]:
df[df["Stock Splits"] > 0]

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Price_SEK,Price_SEK_ydy
Date,,,,,,,,,,
2020-08-31,127.580002,131.0,126.0,129.039993,125.397934,225702700,0.0,4.0,1186.494017,1171.916222


In [60]:
df[df.index >= datetime(2020,8,27)].head()

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Price_SEK,Price_SEK_ydy
Date,,,,,,,,,,
2020-08-27,127.142502,127.485001,123.832497,125.010002,121.481705,155552400,0.0,0.0,1182.425267,1173.474003
2020-08-28,126.012497,126.442497,124.577499,124.807503,121.284912,187630000,0.0,0.0,1171.916222,1182.425267
2020-08-31,127.580002,131.000000,126.000000,129.039993,125.397934,225702700,0.0,4.0,1186.494017,1171.916222
2020-09-01,132.759995,134.800003,130.529999,134.179993,130.392838,151948100,0.0,0.0,1234.667949,1186.494017
2020-09-02,137.589996,137.979996,127.000000,131.399994,127.691307,200119000,0.0,0.0,1279.586966,1234.667949
